In [168]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lego-minifigures-classification/index.csv
/kaggle/input/lego-minifigures-classification/LICENSE
/kaggle/input/lego-minifigures-classification/metadata.csv
/kaggle/input/lego-minifigures-classification/test.csv
/kaggle/input/lego-minifigures-classification/star-wars/0007/006.jpg
/kaggle/input/lego-minifigures-classification/star-wars/0007/002.jpg
/kaggle/input/lego-minifigures-classification/star-wars/0007/001.jpg
/kaggle/input/lego-minifigures-classification/star-wars/0007/007.jpg
/kaggle/input/lego-minifigures-classification/star-wars/0007/005.jpg
/kaggle/input/lego-minifigures-classification/star-wars/0007/004.jpg
/kaggle/input/lego-minifigures-classification/star-wars/0007/003.jpg
/kaggle/input/lego-minifigures-classification/star-wars/0007/008.jpg
/kaggle/input/lego-minifigures-classification/star-wars/0006/006.jpg
/kaggle/input/lego-minifigures-classification/star-wars/0006/002.jpg
/kaggle/input/lego-minifigures-classification/star-wars/0006/010.jpg
/kaggle/input/leg

Imoprting Libraries

In [169]:
import tensorflow as tf # These lib will be useful in getting the APIs for tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
import os # for creating directories, os related stuff
import math
import shutil # to shuffle the images in the dir
import random # again to shufle I guess, not sure!
import matplotlib.pyplot as plt # to plt
import seaborn as sns # to plt
from keras.preprocessing.image import load_img, img_to_array # load image from the dirs
from glob import glob #get the paths which has some pattern to follow
import cv2

In [170]:
data = pd.read_csv('../input/lego-minifigures-classification/index.csv')
data.head(10)

,path,class_id
0,marvel/0001/001.jpg,1
1,marvel/0001/002.jpg,1
2,marvel/0001/003.jpg,1
3,marvel/0001/004.jpg,1
4,marvel/0001/005.jpg,1
5,marvel/0001/006.jpg,1
6,marvel/0001/007.jpg,1
7,marvel/0001/008.jpg,1
8,marvel/0001/009.jpg,1
9,marvel/0001/010.jpg,1


In [171]:
for path in data['path']:
    img = cv2.imread(str('../input/lego-minifigures-classification'+'/'+path)) #Just reading the path to get the very first image
    print(np.array(img)/255)
    break

[[[0.26666667 0.33333333 0.36862745]
  [0.26666667 0.33333333 0.36862745]
  [0.2745098  0.34117647 0.37647059]
  ...
  [0.48235294 0.54901961 0.56078431]
  [0.4745098  0.54117647 0.55294118]
  [0.47058824 0.5372549  0.54901961]]

 [[0.27843137 0.34901961 0.37647059]
  [0.27843137 0.34901961 0.37647059]
  [0.28627451 0.35294118 0.38823529]
  ...
  [0.48627451 0.55294118 0.56470588]
  [0.48235294 0.54901961 0.56078431]
  [0.48235294 0.54901961 0.56078431]]

 [[0.29803922 0.36862745 0.39607843]
  [0.29411765 0.36470588 0.39215686]
  [0.30588235 0.37254902 0.40784314]
  ...
  [0.49019608 0.55686275 0.56862745]
  [0.49019608 0.55686275 0.56862745]
  [0.49019608 0.55686275 0.56862745]]

 ...

 [[0.23529412 0.2745098  0.31372549]
  [0.2        0.23921569 0.27843137]
  [0.18431373 0.22745098 0.25882353]
  ...
  [0.47058824 0.51764706 0.5254902 ]
  [0.49411765 0.54509804 0.55294118]
  [0.45882353 0.50980392 0.51764706]]

 [[0.21960784 0.25882353 0.29803922]
  [0.18039216 0.21960784 0.25882353]


In [172]:
X = []
y = []

for path in data['path']:
    img = cv2.imread(str('../input/lego-minifigures-classification'+'/'+path))
    resized_img = cv2.resize(img,(224,224)) # Resizing the image
    scaled_img = np.array(resized_img)/255 # Scaling the image
    X.append(scaled_img) # Appending it to X

In [173]:
y = np.array(data['class_id'])-1

In [174]:
X = np.array(X)
y = np.array(y)

In [175]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,random_state=0)

In [176]:
data_augmentation = Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal',input_shape = (224,224,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
])

### Model Building

In [177]:
num_classes = data['class_id'].nunique()

In [178]:
model = Sequential([
    data_augmentation,
    layers.Conv2D(64,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16,3,padding='same',activation='relu'),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(num_classes),
])

In [179]:
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), # since we did not use any activation in the output layer
    optimizer = 'adam',
    metrics = ['accuracy'],
)

In [180]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=50,verbose=2)

Epoch 1/50
9/9 - 22s - loss: 3.8824 - accuracy: 0.0341 - val_loss: 3.5940 - val_accuracy: 0.0227
Epoch 2/50
9/9 - 21s - loss: 3.5878 - accuracy: 0.0227 - val_loss: 3.5922 - val_accuracy: 0.0227
Epoch 3/50
9/9 - 21s - loss: 3.5744 - accuracy: 0.0114 - val_loss: 3.5930 - val_accuracy: 0.0341
Epoch 4/50
9/9 - 20s - loss: 3.5522 - accuracy: 0.0644 - val_loss: 3.7783 - val_accuracy: 0.0000e+00
Epoch 5/50
9/9 - 21s - loss: 3.4932 - accuracy: 0.0682 - val_loss: 3.6997 - val_accuracy: 0.0341
Epoch 6/50
9/9 - 20s - loss: 3.3729 - accuracy: 0.0871 - val_loss: 3.7599 - val_accuracy: 0.0682
Epoch 7/50
9/9 - 21s - loss: 3.2151 - accuracy: 0.1553 - val_loss: 3.9129 - val_accuracy: 0.0795
Epoch 8/50
9/9 - 21s - loss: 2.9433 - accuracy: 0.2045 - val_loss: 3.8755 - val_accuracy: 0.1364
Epoch 9/50
9/9 - 21s - loss: 2.6136 - accuracy: 0.3182 - val_loss: 3.6110 - val_accuracy: 0.1932
Epoch 10/50
9/9 - 21s - loss: 2.2775 - accuracy: 0.3750 - val_loss: 3.7353 - val_accuracy: 0.1591
Epoch 11/50
9/9 - 21s - l